In [16]:
import function.functions as functions
# import function.functions_nocuda as functions # use this if you can't use cuda and the GPU

from qutip import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [17]:
# Set device and parameters in this part

############################ Select the device ##############################
# parameters of the device can be set in the parameters folder
device = 'flopping_charge' # change here to change the device: charge_qubit, flopping_spin, flopping_charge, transmon, fluxonium

data = np.load('data/params/'+device+'.npz', allow_pickle=True)
H_sys, drive_op, wq, g, kappa, num_A, dim = data['H_sys'], data['drive_op'], data['wq'], data['g'], data['kappa'], data['num_A'], data['dim']
#############################################################################

########################### Parameters of the paper ###########################
if device == 'charge_qubit':
    Alist = np.linspace(0, 4*wq, 100) # list of A values, should start from 0
                                      # 0,1 for w_r = 0.42
                                      # 0,4 for w_r = 0.8, 1.3

    w_r = 1.3*wq # resonator frequency for longitudinal readout
                 # in the paper : 0.42, 0.8, 1.3

    N_replicas = [-4,-3,-2,-1,0,1,2,3,4]

elif device == 'flopping_spin':
    Alist = np.linspace(0, 0.8, 100) # list of A values, should start from 0
    w_r = 1.4*wq # resonator frequency for longitudinal readout
                 # in the paper : 0.7, 1.4

    H_sys_notsc_notsf, H_sys_notsf, H_ref = Qobj(data['H_sys_notsc_notsf']), Qobj(data['H_sys_notsf']), Qobj(data['H_ref'])

    N_replicas = [0]

elif device == 'flopping_charge':
    Alist = np.linspace(0, 0.3, 100) # list of A values, should start from 0
    w_r = 1.4*wq # resonator frequency for longitudinal readout
                 # in the paper : 0.7, 1.4

    H_sys_notsc_notsf, H_sys_notsf, H_ref = Qobj(data['H_sys_notsc_notsf']), Qobj(data['H_sys_notsf']), Qobj(data['H_ref'])

    N_replicas = [0]

elif device == 'transmon':
    Alist = np.linspace(0, 0.05, 100) # list of A values, should start from 0
    w_r = 1.14*wq # resonator frequency for longitudinal readout
                  # in the paper : 0.77, 1.14

    Ejeff, EC = data['Ejeff'], data['EC']

    N_replicas = [0]

elif device == 'fluxonium':
    Alist = np.linspace(0, 0.35, 100) # list of A values, should start from 0
    w_r = 1.92*wq # resonator frequency for longitudinal readout
                  # in the paper : 0.89, 1.92

    N_replicas = [0]

################################################################################

########################### Custom parameters #################################
# test other parameters Alist, w_r

save_file = True # test mode or save data
################################################################################

In [18]:
def get_color(colorRGBA1, colorRGBA2):
    alpha = 1 - ((1 - colorRGBA1[3]) * (1 - colorRGBA2[3]) / 255)
    red   = (colorRGBA1[0] * (1 - colorRGBA2[3]) + colorRGBA2[0] * colorRGBA2[3]) / 1
    green = (colorRGBA1[1] * (1 - colorRGBA2[3]) + colorRGBA2[1] * colorRGBA2[3]) / 1
    blue  = (colorRGBA1[2] * (1 - colorRGBA2[3]) + colorRGBA2[2] * colorRGBA2[3]) / 1
    return (np.round(red,3), np.round(green,3), np.round(blue,3), np.round(alpha,3))

from matplotlib.colors import LinearSegmentedColormap
cmap=LinearSegmentedColormap.from_list(name='rbb',
	colors=['dodgerblue','white','tab:red'])

def colorFromBivariateData1(Z1,Z2,cmap1 = cmap):
    norm = mpl.colors.Normalize(vmin=0, vmax=1)

    cmap_1 = mpl.cm.ScalarMappable(norm=norm, cmap=cmap1)
    cmap_1.set_array([])
    
    Z1_color = np.array(cmap_1.to_rgba(Z1))
    
    Z1_color[3] = np.round(Z1_color[3]*(1-Z2),3)
    
    return Z1_color

from matplotlib.colors import ListedColormap

my_cmap=LinearSegmentedColormap.from_list(name='rbb2',
	colors=['purple','#2ab07f','gold'])

def colorFromBivariateData2(Z1,Z2,cmap2 = my_cmap):    
    norm = mpl.colors.Normalize(vmin=0, vmax=1)

    cmap_2 = mpl.cm.ScalarMappable(norm=norm, cmap=cmap2)
    cmap_2.set_array([])
    
    Z2_color = np.array(cmap_2.to_rgba(Z1))
        
    Z2_color[3] = np.round(Z2_color[3]*Z2,3)
    
    return Z2_color

In [19]:
def generate_floquet_spectrum(H_sys,drive_op,n_states,N_rep,N_replicas,device):
    H_sys = Qobj(H_sys[0:n_states,0:n_states])
    drive_op = Qobj(drive_op[0:n_states,0:n_states])

    if save_file:
        fname = 'data/'+device+'/floquet_spectrum/w_r='+str(np.round(w_r/wq,3))+'_N_rep='+str(N_rep)+'_n_states='+str(n_states)\
                +'_dim='+str(dim)
        
        data = open(fname+'.txt',"w")
        data.close()
        
    else:
        fname = None

    evals_list = []

    A_list_replica = np.concatenate((-np.array(Alist[::-1])[:-1],np.array(Alist)))

    for ii in N_replicas:
        AA, evals, evecs = functions.construct_eigenvalues_eigenstates_floquet_list(N_rep,Alist,num_A,H_sys,drive_op,w_r,n_states,ii,fname)
        
        evals = np.concatenate(((evals[::-1]).get()[:-1],(evals).get()))

        if ii == 0:
            g_parallel = np.gradient(evals, A_list_replica, axis=0)

            if device == 'flopping_spin' or device == 'flopping_charge':
                ref = functions.construct_eigenstates_nofloquet_noqutip(H_ref, n_states)[1]

                weights_charge = np.zeros((n_states,len(Alist)))
                weights_spin = np.zeros((n_states,len(Alist)))

                color_m_old = np.zeros((4,len(Alist)),dtype=object) # on the row ground/excited state
                neg_color_m_old = np.zeros((4,len(Alist)),dtype=object) # on the row ground/excited state

                for idx, A_q in enumerate(Alist):
                    weight, renormalization = functions.compute_weights(N_rep, ref, evecs[idx])
                
                    weights_charge[:,idx] = np.real(weight[0,:]+weight[1,:])/renormalization
                    weights_spin[:,idx] = np.real(weight[0,:]+weight[2,:])/renormalization
            
                    for i in range(4):
                        color_m_old[i,idx] = get_color(colorFromBivariateData1(weights_charge[i,idx],\
                                                                        weights_spin[i,idx]),\
                                                colorFromBivariateData2(weights_charge[i,idx],weights_spin[i,idx]))
                        neg_color_m_old[i,idx] = get_color(colorFromBivariateData1(1-weights_charge[i,idx],\
                                                                        weights_spin[i,idx]),\
                                                colorFromBivariateData2(1-weights_charge[i,idx],weights_spin[i,idx]))

                color_m = np.zeros((4,len(A_list_replica)),dtype=object) # on the row ground/excited state
                
                for i in range(4):
                    color_m[i,:] = np.concatenate((np.array(neg_color_m_old[i,::-1])[:-1],np.array(color_m_old[i,:])))

        evals_list.append(evals)

    evals_analytical = np.zeros((len(A_list_replica), n_states))
    evals_floquet_analytical = np.zeros((len(A_list_replica), 2))

    if device == 'transmon':
        n01_modulo_squared = 1/2*np.sqrt(Ejeff/(8*EC))
        for j in range(n_states):
            evals_analytical[:, j] = (-Ejeff + np.sqrt(8*Ejeff*EC)*(j+1/2) - EC/12*(6*j**2 + 6*j + 3))*np.ones(len(A_list_replica))
        evals_floquet_analytical[:, 0] = evals_analytical[:,0] - n01_modulo_squared*A_list_replica**2/2*wq/(wq**2 - w_r**2)
        evals_floquet_analytical[:, 1] = evals_analytical[:,1] + n01_modulo_squared*A_list_replica**2/2*(wq/(wq**2 - w_r**2)-2*(wq-EC)/((wq-EC)**2-w_r**2))

    if save_file:
        if device == 'flopping_spin' or device == 'flopping_charge':
            np.savez(fname, A_list_replica=A_list_replica, wq=wq, w_r=w_r, evals_list=evals_list, N_replicas=N_replicas, color_m=color_m)
        elif device == 'transmon':
            np.savez(fname, A_list_replica=A_list_replica, wq=wq, w_r=w_r, evals_list=evals_list, N_replicas=N_replicas,\
                evals_floquet_analytical=evals_floquet_analytical, evals_analytical=evals_analytical)
        else:
            np.savez(fname, A_list_replica=A_list_replica, wq=wq, w_r=w_r, evals_list=evals_list, N_replicas=N_replicas)

In [20]:
n_states_list = [4]
N_rep_list = [15] # this means that we will have 2*N_rep+1 replicas

for n_states in n_states_list:
    for N_rep in N_rep_list:
        generate_floquet_spectrum(H_sys,drive_op,n_states,N_rep,N_replicas,device)